## Example 3: Multi-Product Production and Distribution Problem

This example demonstrates a complex optimization problem involving:
- **Multiple production facilities** (A and B)
- **Multiple products** (represented by index combinations)
- **Transportation** between places
- **Capacity constraints** with efficiency factors

### Problem Description

**Decision Variables:**
- `x[i,j,k,m]`: Production quantity from facility i, destination j, for product type k
  - i ∈ {A, B} (facilities)
  - j ∈ {1, 2, 3, 4, 5} (destination)
  - k ∈ {1, 2, 3} (product type)
  
- `y[i1,i2]`: Transportation flow from facility i1 to destination i2
  - Specifically: y[i1,i2]

**Objective Function:**
Maximize total profit = Revenue - Transportation Costs - Production Costs

- **Revenue Terms:**
  - 1550 × Σ x[i,j,1] (Product type 1)
  - 1600 × Σ x[i,j,2] (Product type 2)
  - 1400 × Σ x[i,j,3] (Product type 3)

- **Transportation Costs:**
  - Destination 1: 10000 + 122 × y[i,j]
  - Destination 2: 12000 + 122 × y[i,j]
  - Destination 3: 80000  + 122 × y[i,j]
  - Destination 4: 70000  + 122 × y[i,j]
  - Destination 5: 90000  + 122 × y[i,j]

- **Production Costs:**
  - Facility A, Product 1: 250 × Σ x[A,j,1]
  - Facility A, Product 2: 320 × Σ x[A,j,2]
  - Facility A, Product 3: 420 × Σ x[A,j,3]
  - Facility B, Product 1: 260 × Σ x[B,j,1]
  - Facility B, Product 2: 210 × Σ x[B,j,2]
  - Facility B, Product 3: 350 × Σ x[B,j,3]

- **Demand for Each Destination(p1, p2, p3)(max):**
  - Destination 1: 400, 250, 100
  - Destination 2: 700, 390, 250
  - Destination 3: 200, 400, 340
  - Destination 4: 800, 400, 400
  - Destination 5: 900, 250, 1130

- **Max Capacity for Each Facility:**
  - Facility A, Product 1: 1200
  - Facility A, Product 2: 1100
  - Facility A, Product 3: 1200
  - Facility B, Product 1: 1400
  - Facility B, Product 2: 1300
  - Facility B, Product 3: 700


### Step 1: Import Required Libraries

We'll use Pyomo for optimization modeling and GLPK (or another solver) to solve the problem.

In [80]:
from pyomo.environ import *

# Display message
print("Pyomo imported successfully!")

Pyomo imported successfully!


### Step 2: Create the Model and Define Sets

Sets are collections of indices that we'll use to organize our decision variables and constraints.

In [81]:
# Create a concrete model
model = ConcreteModel(name="Multi-Product Production and Distribution")

# Define Sets
model.I = Set(initialize=['A', 'B'], doc='Production Facilities')
model.J = Set(initialize=[1, 2, 3, 4, 5], doc='Destinations')
model.K = Set(initialize=[1, 2, 3], doc='Product Types')

print("Sets defined:")
print(f"Facilities (I): {list(model.I)}")
print(f"Destinations (J): {list(model.J)}")
print(f"Product Types (K): {list(model.K)}")

Sets defined:
Facilities (I): ['A', 'B']
Destinations (J): [1, 2, 3, 4, 5]
Product Types (K): [1, 2, 3]


### Step 3: Define Parameters

Parameters are the constant values (data) in our problem. We'll use dictionaries to store this information.

In [82]:
# Revenue per product type
revenue = {1: 1550, 2: 1600, 3: 1400}
model.Revenue = Param(model.K, initialize=revenue, doc='Revenue per unit for each product type')

# Transportation fixed costs per destination
trans_fixed_cost = {1: 10000, 2: 12000, 3: 80000, 4: 70000, 5: 90000}
model.TransFixedCost = Param(model.J, initialize=trans_fixed_cost, doc='Fixed transportation cost per destination')

# Transportation variable cost (per unit)
model.TransVarCost = Param(initialize=122, doc='Variable transportation cost per unit')

# Production costs: facility x product type
prod_cost = {
    ('A', 1): 250, ('A', 2): 320, ('A', 3): 420,
    ('B', 1): 260, ('B', 2): 210, ('B', 3): 350
}
model.ProdCost = Param(model.I, model.K, initialize=prod_cost, doc='Production cost per unit')

# Maximum demand at each destination for each product type
demand = {
    (1, 1): 400, (1, 2): 250, (1, 3): 100,
    (2, 1): 700, (2, 2): 390, (2, 3): 250,
    (3, 1): 200, (3, 2): 400, (3, 3): 340,
    (4, 1): 800, (4, 2): 400, (4, 3): 400,
    (5, 1): 900, (5, 2): 250, (5, 3): 1130
}
model.Demand = Param(model.J, model.K, initialize=demand, doc='Maximum demand per destination and product')

# Maximum production capacity: facility x product type
capacity = {
    ('A', 1): 1200, ('A', 2): 1100, ('A', 3): 1200,
    ('B', 1): 1400, ('B', 2): 1300, ('B', 3): 700
}
model.Capacity = Param(model.I, model.K, initialize=capacity, doc='Maximum production capacity')

print("Parameters defined successfully!")
print("\nSample data:")
print(f"Revenue for product 1: ${model.Revenue[1]}")
print(f"Production cost at facility A for product 1: ${model.ProdCost['A', 1]}")
print(f"Demand at destination 1 for product 1: {model.Demand[1, 1]} units")

Parameters defined successfully!

Sample data:
Revenue for product 1: $1550
Production cost at facility A for product 1: $250
Demand at destination 1 for product 1: 400 units


### Step 4: Define Decision Variables

Decision variables are what the optimizer will determine to maximize profit.

In [83]:
# x[i,j,k] = Production quantity from facility i to destination j for product k
model.x = Var(model.I, model.J, model.K, domain=NonNegativeReals, 
              doc='Production quantity from facility i to destination j for product k')

# y[i,j] = Total transportation flow from facility i to destination j (across all products)
model.y = Var(model.I, model.J, domain=NonNegativeReals, 
              doc='Total transportation from facility i to destination j')

print("Decision variables defined:")
print("- x[i,j,k]: Production quantities (continuous, non-negative)")
print("- y[i,j]: Transportation flows (continuous, non-negative)")

Decision variables defined:
- x[i,j,k]: Production quantities (continuous, non-negative)
- y[i,j]: Transportation flows (continuous, non-negative)


### Step 5: Define the Objective Function

The objective is to maximize total profit = Revenue - Transportation Costs - Production Costs

In [84]:
def objective_rule(model):
    # Calculate total revenue
    revenue_total = sum(model.Revenue[k] * model.x[i, j, k] 
                       for i in model.I for j in model.J for k in model.K)
    
    # Calculate total transportation costs (fixed + variable)
    # Note: We need to pay fixed cost once per destination if we ship there
    trans_cost_total = sum(model.TransFixedCost[j] + model.TransVarCost * model.y[i, j]
                          for i in model.I for j in model.J)
    
    # Calculate total production costs
    prod_cost_total = sum(model.ProdCost[i, k] * model.x[i, j, k]
                         for i in model.I for j in model.J for k in model.K)
    
    # Profit = Revenue - Transportation Costs - Production Costs
    return revenue_total - trans_cost_total - prod_cost_total

model.profit = Objective(rule=objective_rule, sense=maximize, doc='Maximize total profit')

print("Objective function defined: Maximize Profit")

Objective function defined: Maximize Profit


### Step 6: Define Constraints

Now we add all the constraints that limit our production and distribution decisions.

In [85]:
# Constraint 1: Demand satisfaction
# Total production sent to each destination for each product cannot exceed demand
def demand_constraint_rule(model, j, k):
    return sum(model.x[i, j, k] for i in model.I) <= model.Demand[j, k]

model.demand_constraint = Constraint(model.J, model.K, rule=demand_constraint_rule,
                                    doc='Production cannot exceed demand at each destination')

print("Constraint 1 added: Demand satisfaction")
print(f"  Total constraints: {len(model.J) * len(model.K)} = {len(list(model.J))} destinations × {len(list(model.K))} products")

Constraint 1 added: Demand satisfaction
  Total constraints: 15 = 5 destinations × 3 products


In [86]:
# Constraint 2: Production capacity
# Total production at each facility for each product cannot exceed capacity
def capacity_constraint_rule(model, i, k):
    return sum(model.x[i, j, k] for j in model.J) <= model.Capacity[i, k]

model.capacity_constraint = Constraint(model.I, model.K, rule=capacity_constraint_rule,
                                      doc='Production cannot exceed facility capacity')

print("Constraint 2 added: Production capacity")
print(f"  Total constraints: {len(model.I) * len(model.K)} = {len(list(model.I))} facilities × {len(list(model.K))} products")

Constraint 2 added: Production capacity
  Total constraints: 6 = 2 facilities × 3 products


In [87]:
# Constraint 3: Transportation flow definition
# y[i,j] must equal the total products shipped from facility i to destination j
def transportation_flow_rule(model, i, j):
    return model.y[i, j] == sum(model.x[i, j, k] for k in model.K)

model.transportation_flow = Constraint(model.I, model.J, rule=transportation_flow_rule,
                                      doc='Define transportation flow as sum of all products')

print("Constraint 3 added: Transportation flow definition")
print(f"  Total constraints: {len(model.I) * len(model.J)} = {len(list(model.I))} facilities × {len(list(model.J))} destinations")

Constraint 3 added: Transportation flow definition
  Total constraints: 10 = 2 facilities × 5 destinations


### Step 7: Display Model Summary

Before solving, let's review the model structure.

In [88]:
print("=" * 60)
print("MODEL SUMMARY")
print("=" * 60)
print(f"\nModel Name: {model.name}")
print(f"\nSets:")
print(f"  - Facilities (I): {len(model.I)} elements")
print(f"  - Destinations (J): {len(model.J)} elements")
print(f"  - Product Types (K): {len(model.K)} elements")

print(f"\nDecision Variables:")
print(f"  - x[i,j,k]: {len(model.I) * len(model.J) * len(model.K)} variables")
print(f"  - y[i,j]: {len(model.I) * len(model.J)} variables")
print(f"  - Total variables: {len(model.I) * len(model.J) * len(model.K) + len(model.I) * len(model.J)}")

print(f"\nConstraints:")
print(f"  - Demand constraints: {len(model.J) * len(model.K)}")
print(f"  - Capacity constraints: {len(model.I) * len(model.K)}")
print(f"  - Transportation flow: {len(model.I) * len(model.J)}")
print(f"  - Total constraints: {len(model.J) * len(model.K) + len(model.I) * len(model.K) + len(model.I) * len(model.J)}")

print(f"\nObjective: Maximize Profit")
print("=" * 60)

MODEL SUMMARY

Model Name: Multi-Product Production and Distribution

Sets:
  - Facilities (I): 2 elements
  - Destinations (J): 5 elements
  - Product Types (K): 3 elements

Decision Variables:
  - x[i,j,k]: 30 variables
  - y[i,j]: 10 variables
  - Total variables: 40

Constraints:
  - Demand constraints: 15
  - Capacity constraints: 6
  - Transportation flow: 10
  - Total constraints: 31

Objective: Maximize Profit


### Step 8: Solve the Model

We'll use a solver (GLPK is free and commonly used) to find the optimal solution.

In [89]:
# Create solver instance
solver = SolverFactory('glpk')  # You can also use 'gurobi', 'cplex', 'cbc', etc.

# Solve the model
print("Solving the optimization problem...")
print("This may take a moment...\n")

results = solver.solve(model, tee=True)  # tee=True shows solver output

# Check solver status
print("\n" + "=" * 60)
print("SOLVER STATUS")
print("=" * 60)
print(f"Solver Status: {results.solver.status}")
print(f"Termination Condition: {results.solver.termination_condition}")
print("=" * 60)

Solving the optimization problem...
This may take a moment...

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp5a2kxcmc.glpk.raw --wglp /tmp/tmpyvnzqgwv.glpk.glp --cpxlp
 /tmp/tmp1old_xtm.pyomo.lp
Reading problem data from '/tmp/tmp1old_xtm.pyomo.lp'...
31 rows, 41 columns, 100 non-zeros
284 lines were read
Writing problem data to '/tmp/tmpyvnzqgwv.glpk.glp'...
238 lines were written
GLPK Simplex Optimizer 5.0
31 rows, 41 columns, 100 non-zeros
Preprocessing...
21 rows, 30 columns, 60 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 21
*     0: obj =  -5.240000000e+05 inf =   0.000e+00 (30)
*    18: obj =   6.304020000e+06 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 Mb (66933 bytes)
Writing basic solution to '/tmp/tmp5a2kxcmc.glpk.raw'...
81 lines were written
GLPSOL--

### Step 9: Display Optimal Solution

Let's examine the optimal production and distribution plan.

In [90]:
if results.solver.termination_condition == TerminationCondition.optimal:
    print("\n" + "=" * 60)
    print("OPTIMAL SOLUTION FOUND!")
    print("=" * 60)
    
    # Display optimal profit
    print(f"\n💰 Maximum Profit: ${value(model.profit):,.2f}")
    print("=" * 60)
    
    # Display production plan (only non-zero values)
    print("\n📦 PRODUCTION PLAN (x[i,j,k]):")
    print("-" * 60)
    print(f"{'Facility':<10} {'Dest':<6} {'Product':<8} {'Quantity':<10}")
    print("-" * 60)
    
    for i in model.I:
        for j in model.J:
            for k in model.K:
                qty = value(model.x[i, j, k])
                if qty > 0.01:  # Only show non-zero values
                    print(f"{i:<10} {j:<6} {k:<8} {qty:>10.2f}")
    
    print("-" * 60)
    
    # Display transportation flows
    print("\n🚚 TRANSPORTATION FLOWS (y[i,j]):")
    print("-" * 60)
    print(f"{'From Facility':<15} {'To Destination':<18} {'Total Units':<12}")
    print("-" * 60)
    
    for i in model.I:
        for j in model.J:
            flow = value(model.y[i, j])
            if flow > 0.01:  # Only show non-zero values
                print(f"{i:<15} {j:<18} {flow:>12.2f}")
    
    print("-" * 60)
    
else:
    print("\n⚠️ WARNING: Optimal solution not found!")
    print(f"Termination condition: {results.solver.termination_condition}")


OPTIMAL SOLUTION FOUND!

💰 Maximum Profit: $6,304,020.00

📦 PRODUCTION PLAN (x[i,j,k]):
------------------------------------------------------------
Facility   Dest   Product  Quantity  
------------------------------------------------------------
A          1      1            400.00
A          2      1            700.00
A          3      1            100.00
A          4      2            140.00
A          4      3            390.00
A          5      2            250.00
A          5      3            810.00
B          1      2            250.00
B          1      3            100.00
B          2      2            390.00
B          2      3            250.00
B          3      1            100.00
B          3      2            400.00
B          3      3            340.00
B          4      1            800.00
B          4      2            260.00
B          4      3             10.00
B          5      1            500.00
------------------------------------------------------------

🚚 TRA

### Step 10: Analyze Production by Facility

Let's see how much each facility produces for each product type.

In [91]:
if results.solver.termination_condition == TerminationCondition.optimal:
    print("\n" + "=" * 60)
    print("PRODUCTION ANALYSIS BY FACILITY")
    print("=" * 60)
    
    for i in model.I:
        print(f"\n🏭 Facility {i}:")
        print("-" * 40)
        for k in model.K:
            total_prod = sum(value(model.x[i, j, k]) for j in model.J)
            capacity_used = (total_prod / model.Capacity[i, k]) * 100
            print(f"  Product {k}:")
            print(f"    Production: {total_prod:>8.2f} units")
            print(f"    Capacity:   {model.Capacity[i, k]:>8.0f} units")
            print(f"    Utilization: {capacity_used:>6.1f}%")
            print()


PRODUCTION ANALYSIS BY FACILITY

🏭 Facility A:
----------------------------------------
  Product 1:
    Production:  1200.00 units
    Capacity:       1200 units
    Utilization:  100.0%

  Product 2:
    Production:   390.00 units
    Capacity:       1100 units
    Utilization:   35.5%

  Product 3:
    Production:  1200.00 units
    Capacity:       1200 units
    Utilization:  100.0%


🏭 Facility B:
----------------------------------------
  Product 1:
    Production:  1400.00 units
    Capacity:       1400 units
    Utilization:  100.0%

  Product 2:
    Production:  1300.00 units
    Capacity:       1300 units
    Utilization:  100.0%

  Product 3:
    Production:   700.00 units
    Capacity:        700 units
    Utilization:  100.0%



### Step 11: Analyze Demand Satisfaction

Let's check how much demand is satisfied at each destination.

In [92]:
if results.solver.termination_condition == TerminationCondition.optimal:
    print("\n" + "=" * 60)
    print("DEMAND SATISFACTION ANALYSIS")
    print("=" * 60)
    
    for j in model.J:
        print(f"\n📍 Destination {j}:")
        print("-" * 40)
        for k in model.K:
            total_supplied = sum(value(model.x[i, j, k]) for i in model.I)
            demand_satisfaction = (total_supplied / model.Demand[j, k]) * 100
            print(f"  Product {k}:")
            print(f"    Supplied: {total_supplied:>8.2f} units")
            print(f"    Demand:   {model.Demand[j, k]:>8.0f} units")
            print(f"    Satisfied: {demand_satisfaction:>6.1f}%")
            print()


DEMAND SATISFACTION ANALYSIS

📍 Destination 1:
----------------------------------------
  Product 1:
    Supplied:   400.00 units
    Demand:        400 units
    Satisfied:  100.0%

  Product 2:
    Supplied:   250.00 units
    Demand:        250 units
    Satisfied:  100.0%

  Product 3:
    Supplied:   100.00 units
    Demand:        100 units
    Satisfied:  100.0%


📍 Destination 2:
----------------------------------------
  Product 1:
    Supplied:   700.00 units
    Demand:        700 units
    Satisfied:  100.0%

  Product 2:
    Supplied:   390.00 units
    Demand:        390 units
    Satisfied:  100.0%

  Product 3:
    Supplied:   250.00 units
    Demand:        250 units
    Satisfied:  100.0%


📍 Destination 3:
----------------------------------------
  Product 1:
    Supplied:   200.00 units
    Demand:        200 units
    Satisfied:  100.0%

  Product 2:
    Supplied:   400.00 units
    Demand:        400 units
    Satisfied:  100.0%

  Product 3:
    Supplied:   340.

### Step 12: Cost and Revenue Breakdown

Finally, let's break down the components of our profit.

In [93]:
if results.solver.termination_condition == TerminationCondition.optimal:
    print("\n" + "=" * 60)
    print("FINANCIAL BREAKDOWN")
    print("=" * 60)
    
    # Calculate total revenue
    total_revenue = sum(model.Revenue[k] * value(model.x[i, j, k]) 
                       for i in model.I for j in model.J for k in model.K)
    
    # Calculate transportation costs
    total_trans_cost = sum(model.TransFixedCost[j] + model.TransVarCost * value(model.y[i, j])
                          for i in model.I for j in model.J if value(model.y[i, j]) > 0.01)
    
    # Calculate production costs
    total_prod_cost = sum(model.ProdCost[i, k] * value(model.x[i, j, k])
                         for i in model.I for j in model.J for k in model.K)
    
    print(f"\n💵 Total Revenue:           ${total_revenue:>12,.2f}")
    print(f"📦 Production Costs:        ${total_prod_cost:>12,.2f}")
    print(f"🚚 Transportation Costs:    ${total_trans_cost:>12,.2f}")
    print("-" * 60)
    print(f"💰 NET PROFIT:              ${value(model.profit):>12,.2f}")
    print("=" * 60)
    
    # Profit margin
    profit_margin = (value(model.profit) / total_revenue) * 100
    print(f"\n📊 Profit Margin: {profit_margin:.2f}%")


FINANCIAL BREAKDOWN

💵 Total Revenue:           $9,394,000.00
📦 Production Costs:        $1,810,800.00
🚚 Transportation Costs:    $1,279,180.00
------------------------------------------------------------
💰 NET PROFIT:              $6,304,020.00

📊 Profit Margin: 67.11%
